# Imports

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from PIL import Image

class_names = ["Plane", "Car", "Bird", "Cat", "Deer", "Dog", "Frog", "Horse", "Ship", "Truck"]

# Datasets Loading

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [3]:
train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=True
)

# Device Setting

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model Creation

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):           # x.shape = N, 3, 32, 32
        x = F.relu(self.conv1(x))   # x.shape = N, 16, 30, 30
        x = self.pool(x)            # x.shape = N, 16, 15, 15
        x = F.relu(self.conv2(x))   # x.shape = N, 32, 13, 13
        x = self.pool(x)            # x.shape = N, 32, 6, 6
        x = F.relu(self.conv3(x))   # x.shape = N, 64, 4, 4
        x = self.pool(x)            # x.shape = N, 64, 2, 2
        x = torch.flatten(x, 1)     # x.shape = N, 64 * 2 * 2
        x = F.relu(self.fc1(x))     # x.shape = N, 128
        x = F.relu(self.fc2(x))     # x.shape = N, 64
        x = self.fc3(x)             # x.shape = N, 10
        return x

model = Net().to(device)

# Loss and Optimizer

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001)

# Model Training

In [7]:
model.train()

for epoch in range(5):
    for batch_index, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        loss = loss_fn(output, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (batch_index + 1) % 300 == 0:
            print (f'Epoch [{epoch + 1}], Step [{batch_index + 1}], Loss: {loss.item():.4f}')

Epoch [1], Step [300], Loss: 1.6696
Epoch [1], Step [600], Loss: 1.5611
Epoch [1], Step [900], Loss: 1.6708
Epoch [1], Step [1200], Loss: 1.4674
Epoch [1], Step [1500], Loss: 1.4280
Epoch [2], Step [300], Loss: 1.2632
Epoch [2], Step [600], Loss: 1.2795
Epoch [2], Step [900], Loss: 1.4341
Epoch [2], Step [1200], Loss: 1.1430
Epoch [2], Step [1500], Loss: 1.0752
Epoch [3], Step [300], Loss: 1.3436
Epoch [3], Step [600], Loss: 0.9872
Epoch [3], Step [900], Loss: 1.0184
Epoch [3], Step [1200], Loss: 1.0299
Epoch [3], Step [1500], Loss: 0.9368
Epoch [4], Step [300], Loss: 0.9210
Epoch [4], Step [600], Loss: 0.8720
Epoch [4], Step [900], Loss: 1.0026
Epoch [4], Step [1200], Loss: 0.8217
Epoch [4], Step [1500], Loss: 1.0565
Epoch [5], Step [300], Loss: 0.9378
Epoch [5], Step [600], Loss: 0.8165
Epoch [5], Step [900], Loss: 0.8310
Epoch [5], Step [1200], Loss: 0.8464
Epoch [5], Step [1500], Loss: 1.1023


# Model Evaluation

In [8]:
model.eval()

with torch.no_grad():
    n_samples = len(test_loader.dataset)
    n_correct = 0

    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(targets)

        output = model(data)

        _, predicted = torch.max(output, 1)

        n_correct += (predicted == targets).sum().item()

    accuracy = 100.0 * n_correct / n_samples
    print(f'Accuracy of the Net: {accuracy}%')

Accuracy of the Net: 66.81%


# Model Saving

In [9]:
torch.save(model.state_dict(), 'net.pth')

# Model Loading

In [5]:
model = Net()
model.load_state_dict(torch.load('net.pth'))
model.to(device)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

# Model Testing with Real Images

In [10]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

def load_image(path):
    image = Image.open(path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

images = [
    load_image("test_images/bird.jpg"),
    load_image("test_images/deer.jpg"),
    load_image("test_images/dog.jpg"),
    load_image("test_images/plane.jpg")
]

model.eval()
for image in images:
    output = model(image)
    _, predicted = torch.max(output, 1)
    print(f"Prediction: {class_names[predicted.item()]}")

Prediction: Bird
Prediction: Bird
Prediction: Dog
Prediction: Plane
